# Neural Image Super-Resolution (Local Version)

Modified version of [superresolution_gan](https://github.com/fukumame/superresolution_gan), implementation of [Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network](https://arxiv.org/abs/1609.04802).

This notebook works with local files instead of Google Drive files.

Seems to work best on very small images. Images larger than 200x200 (in total number of pixels) will be sliced, processed in tiles and combined.

In [ ]:
#@title Setup Required Libraries
!pip -q install import-ipynb
!pip -q install image_slicer

import os
import glob
import random
import string

# Create temporary directory for processing
dir_tmp = "/content/tmp/"
if not os.path.exists(dir_tmp):
    os.makedirs(dir_tmp)

# Utility functions from inhagcutils
def path_leaf(path):
    """Get the filename from a path"""
    return os.path.basename(path)

def path_dir(path):
    """Get the directory from a path"""
    return os.path.dirname(path) + "/"

def rnd_str(l):
    return ''.join(random.choice(string.ascii_lowercase) for i in range(l))

def create_dirs(dirs):
    """Create directories if they don't exist"""
    for dir in dirs:
        if not os.path.exists(dir):
            os.makedirs(dir)

### Settings

In [ ]:
#@title Path to a local image file
input_image = "/path/to/your/image.png" #@param {type:"string"}

#@markdown <font color="#999">Output will be saved in the same directory as `superres_<filename>`</font>

dir_out = path_dir(input_image)
output_image = path_dir(input_image) + "superres_" + path_leaf(input_image)

target_grid = 200*200

dir_input_slices = dir_tmp + "input-slices/"
dir_output_slices = dir_tmp + "output-slices/"
create_dirs([dir_tmp, dir_input_slices, dir_output_slices])

### Setup

In [ ]:
#@title Git clone & get model
%cd /content/
!git clone https://github.com/fukumame/superresolution_gan.git
%cd /content/superresolution_gan
!wget https://www.dropbox.com/s/l4s5a6v4licks62/generator_model_3008000.npz
import image_slicer, math
from PIL import Image

### **Action**

In [ ]:
#@title Superres
im = Image.open(input_image)
width, height = im.size

# Tile & save tiles
def slice_image(input_image, dir):
  tiles = closestDiv(math.floor((width*height)/target_grid), 4)
  slicery = image_slicer.slice(input_image, tiles, save=False)
  image_slicer.save_tiles(slicery, directory=dir, prefix='slice')

# Superres tiles
def superres(dir_in, dir_out):
  slices = glob.glob(dir_in+"slice*")
  %cd /content/superresolution_gan/
  i = 1
  for slice in slices:
    output = dir_out+path_leaf(slice)
    print('> processing slice', str(i)+"/"+str(len(slices)))
    !python superresolution.py --modelpath=generator_model_3008000.npz --imagepath="{slice}" --outputpath="{output}"
    i+=1

# Mash
def mash(dir_in, img_out):
  tiles = image_slicer.open_images_in(dir_in)
  image = image_slicer.join(tiles)
  tmpImg = dir_tmp+rnd_str(8)
  image.save('/content/tmp/joined.png')
  !cp "/content/tmp/joined.png" "{output_image}"

# Cleanup
def clean():
  copy = dir_tmp+path_leaf(input_image)
  !rm -f {dir_input_slices}*
  !rm -f {dir_output_slices}*
  !rm -f {dir_tmp}joined.png
  if os.path.exists(copy):
    !rm -f {copy}

def closestDiv(a, b):
  c1 = a - (a % b)
  c2 = (a + b) - (a % b)
  if (a - c1 > c2 - a):
    return c2
  else:
    return c1

if width*height > target_grid:
  # Create a copy in tmp directory for processing
  tmp_copy = os.path.join(dir_tmp, path_leaf(input_image))
  !cp "{input_image}" "{tmp_copy}"
  print(f'Slicing {input_image}...')
  slice_image(tmp_copy, dir_input_slices)
  print('Done.\nSuperres slices...')
  superres(dir_input_slices, dir_output_slices)
  print('Done.\nMashing back into single image...')
  mash(dir_output_slices, output_image)
  print(f'Done\nFile saved as {output_image}')
  clean()
else:
  %cd /content/superresolution_gan
  print(f'Processing {input_image}')
  !python superresolution.py --modelpath=generator_model_3008000.npz --imagepath="{input_image}" --outputpath="{output_image}"
  print(f'Saved to {output_image}')

### Cleanup (Optional)

Run this cell to clean up temporary files when you're done.

In [ ]:
# Remove temporary directory
!rm -rf {dir_tmp}
print("Temporary files cleaned up.")